**_Autor_**: Rubén del Mazo Rodríguez

# Creación de una red neuronal profunda paso a paso

Anteriormente se entrenó una red neuronal con una única capa oculta. En este archivo se quita la última restricción, el número de capas, y se implementan las funciones necesarias para construir una red neuronal profunda con tantas capas y unidades ocultas como se desee. 

Se volverá a crear la red neuronal con una única capa oculta, pero cambiando su estructura y funciones de activación, adaptándolas al problema de clasificación para el que será utilizada. Y las funciones de esta nueva estructura serán aprovechadas para crear el caso general de $L$ capas ocultas. De esta manera, podremos comparar en el siguiente y último archivo la diferencia de rendimiento entre una red con una capa oculta y una red con $L-1$ capas ocultas.

**Puntos principales:**

- Construir las funciones necesarias para una red neuronal profunda con L capas. 
- Utilizar unidades ocultas con la función de activación que faltaba por mostrar, ReLU, la cual da mejores resultados en general.
- Implementar la estructura de la red en una clase fácil de utilizar.

**Notación**:
- El superíndice $[l]$ indica una cantidad o variable asociada con la capa l-ésima. 
    - Por ejemplo: $a^{[L]}$ es la matriz de activación/salida de la capa L. $W^{[L]}$ y $b^{[L]}$ son los parámetros de la capa L, siendo el primero la matriz de pesos y el segundo el vector de sesgos.
- El superíndice $(i)$ indica una cantidad o variable asociada al ejemplo de entrenamiento i-ésimo.
    - Ejemplo: $x^{(i)}$ es el ejemplo de entrenamiento i-ésimo.
- El subíndice $k$ indica la fila k-ésima de un vector o matriz, mientras que el subíndice $j$ señala la columna j-ésima. También pueden indicar el número de unidad oculta o "neurona" en una capa l-ésima.
    - Ejemplo: $a^{[l](i)}_k$ indica el valor de activación correspondiente a la fila (neurona) k-ésima la capa l-ésima del vector de entrenamiento i-ésimo.

## Contenidos
- [1 - Librerías](#1)
- [2 - Esquema general del ciclo de una red neuronal profunda](#2)
- [3 - Modelo de red neuronal de dos capas (una capa oculta)](#3)
    - [3.1 - Inicialización de los parámetros](#3-1)
    - [3.2 - Propagación hacia delante (_forward propagation_)](#3-2)
        - [3.2.1 - Funciones de activación](#3-2-1)
        - [3.2.2 - Cálculo de la función lineal de propagación](#3-2-2)
        - [3.2.3 - Cálculo de las funciones de activación](#3-2-3)
    - [3.3 - Cálculo del coste global](#3-3)
    - [3.4 - Retropropagación (_backward propagation_)](#3-4)
        - [3.4.1 - Derivadas de las funciones de activación](#3-4-1)
        - [3.4.2 - Cálculo de las derivadas de la función lineal](#3-4-2)
        - [3.4.3 - Cálculo de las derivadas de las funciones de activación](#3-4-2)
    - [3.5 - Actualización de los parámetros](#3-5)
- [4 - Modelo de red neuronal con L capas (L-1 capas ocultas)](#4)
    - [4.1 - Inicialización de los parámetros](#4-1)
    - [4.2 - Propagación hacia delante (_forward propagation_)](#4-2)
    - [4.3 - Cálculo del coste global](#4-3)
    - [4.4 - Retropropagación (_backward propagation_)](#4-4)
    - [4.5 - Actualización de los parámetros](#4-5)

<a name='1'></a>
## 1 - Librerías

En primer lugar, importamos y ejecutamos las librerías necesarias:

- [numpy](https://numpy.org/doc/1.24/) es el paquete fundamental para la computación científica con Python.
- `np.random.seed(1)` se utiliza para mantener la coherencia de todas las llamadas a funciones aleatorias. Ya se ha hablado sobre ello en archivos anteriores.

In [1]:
import numpy as np

np.random.seed(1)

<a name='2'></a>
## 2 - Esquema general del ciclo de una red neuronal profunda

Como hasta ahora se ha explicado y mostrado en los archivos anteriores, la construcción de una red neuronal consta de varios pasos y se crearán funciones que cumplan cada uno de ellos. El esquema para su creación es el siguiente:

- Inicialización de los parámetros (tanto para el caso de la red neuronal de dos capas como la de $L$ capas).
- Implementar el módulo de propagación hacia delante (en morado en la figura inferior):
     - Completar la parte LINEAL del paso de propagación hacia delante de la capa, siendo el resultado $Z^{[l]}$.
     - Definir/disponer de la función de ACTIVACIÓN, en este caso, ReLU y sigmoide (`relu()` y `sigmoide()`)
     - Combinar los dos pasos anteriores en la función de activación hacia delante, [LINEAL -> ACTIVACIÓN]
         - En esta red particular, realizaremos la combinación [LINEAL -> RELU] L-1 veces (para las capas 1 a la L-1) y se añade [LINEAL -> SIGMOIDE] al final para la última capa $L$. Con esto tendríamos la propagación hacia delante para un modelo de L capas.
- Calcular la pérdida/coste.
- Implementar el módulo de la retropropagación (en rojo en la figura inferior):
    - Completar la parte LINEAL del paso de retropropagación de la capa.
    - Definir/disponer de las funciones que implementan el GRADIENTE de las funciones de ACTIVACIÓN (`relu_backward()` y `sigmoide_backward()`). 
    - Combinar los dos pasos anteriores en la función de retropropagación, [LINEAL -> ACTIVACION].
        - En esta red particular, realizaremos la combinación [LINEAL -> RELU_backward] L-1 veces (para las capas 1 a la L-1) y se añade [LINEAL -> SIGMOIDE_backward] al final para la última capa $L$. Con esto tendríamos la retropropagación para un modelo de L capas.
- Finalmente, se actualizan los parámetros.

La siguiente imagen muestra esta descripción:

<img src="imagenes/esquema_general.png" style="width:800px;height:500px;">
<caption><center><b>Figura 1.</b> Esquema general del modelo de red neuronal profunda.</center></caption><br>


**Notas de implementación**:

Para cada función de propagación hacia delante, existe una función de retropropagación. Por eso, en cada paso del módulo de propagación hacia delante se almacenarán algunos valores en una "cache". Estos valores almacenados en caché son útiles para calcular gradientes. En el módulo de retropropagación, se puede utilizar la caché para calcular los gradientes.

<a name='3'></a>
## 3 - Modelo de red neuronal de dos capas (una capa oculta)

<a name='3-1'></a>
### 3.1 - Inicialización de los parámetros

En este apartado se crea la función que inicializa los parámetros para la red neuronal de 2 capas (una capa oculta).

- La estructura del modelo es: *LINEAL -> RELU -> LINEAL -> SIGMOIDE*. 
- Se utilizará una inicialización aleatoria de la matriz de pesos: `np.random.randn() * 0.01`. Documentación para [np.random.randn](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html).
- Se utilizará una inicialización con ceros para los sesgos: `np.zeros()`. La documentación para [np.zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html).

> Nota: Es igual que en la red neuronal de una capa oculta implementada en el anterior archivo.

In [2]:
def inicializar_parametros(n_x, n_h, n_y):
    """
    Argumentos:
    n_x -- tamaño de la capa de entrada
    n_h -- tamaño de la capa oculta
    n_y -- tamaño de la capa de salida
    
    Devuelve:
    parametros -- diccionario python que contiene los parámetros:
                    W1 -- matriz de pesos de la forma (n_h, n_x)
                    b1 -- vector de sesgos de la forma (n_h, 1)
                    W2 -- matriz de pesos de la forma (n_y, n_h)
                    b2 -- vector de sesgos de la forma (n_y, 1)                   
    """   
    np.random.seed(1)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape=(n_y, 1))    

    # Asegurar que las dimensiones son las correctas
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parametros = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parametros

<a name='3-2'></a>
### 3.2 - Propagación hacia delante (_forward propagation_)

Una vez inicializados los parámetros, continuamos con el módulo de propagación hacia delante. Para la red neuronal de dos capas se necesitan dos funciones, las cuales se utilizarán luego en el modelo general de L capas:

- LINEAL
- LINEAL -> ACTIVACION donde ACTIVACION es, en esta red, ReLU o sigmoide.

Definamos en primer lugar las funciones de activación:

<a name='3-2-1'></a>
#### 3.2.1 - Funciones de activación

In [3]:
def sigmoide(Z):
    """
    Calcula el sigmoide de Z
    
    Argumentos:
    Z -- array de NumPy de cualquier dimension (salida lineal de la capa)
    
    Devuelve:
    A -- resultado de sigmoide(Z), de las mismas dimensiones que Z.
    cache -- devuelve el argumento de entrada, Z, que se utilizara en la retropropagacion.
    """
    
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    
    return A, cache

In [4]:
def relu(Z):
    """
    Implementa la funcion ReLU.

    Argumentos:
    Z -- array de NumPy de cualquier dimension (salida lineal de la capa)

    Returns:
    A -- resultado de aplicar relu(Z), de las mismas dimensiones que Z. Es el parametro post-activacion.
    cache -- devuelve el argumento de entrada, Z, que se utilizara en la retropropagacion.
    """
    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache

<a name='3-2-2'></a>
#### 3.2.2 - Cálculo de la función lineal de propagación

A continuación se crea la parte lineal de la propagación hacia delante la cual, vectorizada para todos los ejemplos de entrenamiento, sigue la siguiente ecuación.

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{1}$$

donde $A^{[0]} = X$ y $X$ es la matriz de entrenamiento.

In [5]:
def forward_lineal(A, W, b):
    """
    Implementa la parte lineal de la propagación hacia delante de una capa.

    Argumentos:
    A -- activaciones de la capa anterior (o datos de entrada): (tamaño de la capa anterior, numero de ejemplos).
    W -- matriz de pesos: matriz numpy de dimensiones (tamaño de la capa actual, tamaño de la capa anterior).
    b -- vector de sesgos: matriz numpy de dimensiones (tamaño de la capa actual, 1).

    Devuelve:
    Z -- la entrada de la función de activación, tambien llamado parametro de pre-activacion .
    cache -- una tupla python que contiene "A", "W" y "b" ; almacenada para calcular eficientemente la retropropagación.
    """
    
    Z = np.dot(W, A) + b
    # Aseguramos que las dimensiones son correctas
    assert(Z.shape == (W.shape[0], A.shape[1]))
    
    cache = (A, W, b)
    
    return Z, cache

<a name='3-2-3'></a>
##### 3.2.3 - Cálculo de las funciones de activación

En esta red neuronal se utilizan dos funciones de activación:

- **Sigmoide**: Matemáticamente es $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$. Se ha implementado en la función `sigmoide()`.

- **ReLU**: Matemáticamente es $A = ReLU(Z) = max(0, Z)$. Se ha implementado en la función `relu()`.

Ambas funciones devuelven dos variables: el valor de activación, "`A`" y un "`cache`" que contiene "`Z`" (aquello que se proveerá a la función de retropropagación correspondiente). Para usarlas, realizamos las siguientes llamadas:

``` python
A, activacion_cache = sigmoide(Z)

A, activacion_cache = relu(Z)
```

Por comodidad, se agruparán las funciones que calculan el paso lineal y el de activación en una sola función que se definirá a continuación. Su relación matemática para cualquier capa, $l$, en todo el conjunto de entrenamiento es:

$$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})\tag{2}$$

donde la función de activación "g" puede ser `sigmoide()` o `relu()`. Utilizaremos `forward_lineal()` y la correspondiente función de activación.

>**Note**: En el aprendizaje profundo, el cálculo "[LINEAL->ACTIVACION]" cuenta como una única capa en una red neuronal, no como dos capas. Se podría definir todo en la misma función, pero se ha decidido modularizar cada apartado para mejor comprensión y explicación.

In [6]:
def forward_activacion_lineal(A_prev, W, b, activacion):
    """
    Implementar la propagación hacia delante para la capa de la forma LINEAL->ACTIVACIÓN

    Argumentos:
    A_prev -- activaciones de la capa anterior (o datos de entrada): (tamaño de la capa anterior, numero de ejemplos)
    W -- matriz de pesos: matriz numpy de forma (tamaño de la capa actual, tamaño de la capa anterior)
    b -- vector de sesgo, matriz numpy de forma (tamaño de la capa actual, 1)
    activacion -- la funcion de activacion a utilizar en esta capa, almacenada como cadena de texto: "sigmoide" o "relu".

    Devuelve:
    A -- la salida de la funcion de activacion, tambien llamada valor post-activación 
    cache -- una tupla python que contiene "cache_lineal" y "activacion_cache";
             almacenada para calcular la retropropagacion eficientemente
    """
    
    if activacion == "sigmoide":
        Z, cache_lineal = forward_lineal(A_prev, W, b)
        A, activacion_cache = sigmoide(Z)
    elif activacion == "relu":
        Z, cache_lineal = forward_lineal(A_prev, W, b)
        A, activacion_cache = relu(Z)        
    else:
        print("¡Error! Solo se admiten relu o sigmoide como parametros en \"activacion\"")
    
    # Aseguramos dimensiones
    assert(A.shape == (W.shape[0], A_prev.shape[1]))

    cache = (cache_lineal, activacion_cache)

    return A, cache

<a name='3-3'></a>
### 3.3 - Cálculo del coste global

Una vez calculada $A^{L}$ hay que calcular el coste para comprobar que el modelo está aprendiendo los parámetros óptimos. Como ya es habitual, se utilizará la pérdida de entropía cruzada para un problema de clasificación binario:

$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{3}$$

Para esta red neuronal, $L$ = 2.

In [7]:
def calcular_coste(AL, Y):
    """
    Calcula el coste de la entropía cruzada dado en la ecuación (3).

    Argumentos:
    AL -- La salida de la activación de la capa L, de dimensiones (1, numero de casos de entrenamiento).
          Corresponde con el vector de probabilidades asociado a las etiquetas verdaderas, de las mismas dimensiones.
    Y -- Vector de etiquetas verdaderas de dimensiones (1, número de casos de entrenamiento).

    Devuelve:
    coste -- coste de la entropia cruzada dada la ecuacion (3).
    """
    
    # Numero de casos de entrenamiento
    m = Y.shape[1]

    # Calculo de la perdida de entropia cruzada
    # Forma 1
    # perdida = np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL))
    # coste = (-1 / m) * np.sum(perdida)

    # Forma 2
    coste = (1. / m) * (- np.dot(Y, np.log(AL).T) - np.dot(1 - Y, np.log(1 - AL).T))
    coste = np.squeeze(coste) # Por ejemplo, convierte [[17]] en 17
    
    assert(coste.shape == ())

    return coste

<a name='3-4'></a>
### 3.4 - Retropropagación (_backward propagation_)

Tal y como se hizo para el módulo de propagación hacia delante, se implementarán funciones para la retropropagación. Se calculará el gradiente de la función de coste con respecto a los parámetros. El gráfico computacional es:

<img src="imagenes/grafico_computacional.png">
<caption><center><b>Figura 2</b>: Propagación hacia delante junto con la retropropagación para la estructura de cálculo de activaciones [LINEAL -> RELU ($g^{[1]}$) -> LINEAL -> SIGMOIDE ($g^{[2]}$)]. Los bloques negros representan la propagación hacia delante, mientras que los bloques naranjas representa la retropropagación.</center></caption>

La regla de la cadena del cálculo que se utiliza para derivar el coste con respecto a los parámetros es, por ejemplo, para $Z^{[1]}$, en el caso de la red de dos capas:

$$dZ^{[1]} = \frac{d \mathcal{J}(A^{[2]},Y)}{{dZ^{[1]}}} = \frac{d\mathcal{J}(A^{[2]},Y)}{{dA^{[2]}}}\frac{{dA^{[2]}}}{{dZ^{[2]}}}\frac{{dZ^{[2]}}}{{dA^{[1]}}}\frac{{dA^{[1]}}}{{dZ^{[1]}}} \tag{4} $$

Para calcular el gradiente de $dW^{[1]} = \frac{\partial J}{\partial W^{[1]}}$, se utiliza la ecuación (4): $dW^{[1]} = dZ^{[1]} \times \frac{\partial Z^{[1]} }{\partial W^{[1]}}$. Durante la retropropagación, en cada paso se multiplica el gradiente de turno por el gradiente correspondiente a la capa específica para obtener el gradiente deseado.

De forma equivalente, para calcular el gradiente $db^{[1]} = \frac{\partial J}{\partial b^{[1]}}$, se utiliza: $db^{[1]} = dZ^{[1]} \times \frac{\partial Z^{[1]} }{\partial b^{[1]}}$. De ahí que se hable de **retropropagación (_backpropagation_)**.

La construcción de la retropropagación es similar a la de la propagación hacia delante, donde implementaremos dos funciones:

1. LINEAL backward
2. LINEAL -> ACTIVACION backward donde ACTIVACION es, en esta red, la derivada de la función ReLU o sigmoide.

Importante tener en cuenta que:

- `b` es una matriz (np.ndarray) con una columna y $n^{[l]}_h$ filas. Por ejemplo: b = [[1.0], [2.0]].
- `np.sum()` realiza la suma sobre todos los elementos del ndarray.
- axis = 1 o axis = 0 especifica si la suma se realiza por filas o por columnas, respectivamente.
- _keepdims_ especifica si las dimensiones originales de la matriz se mantienen o no. Es decir, por ejemplo, si la matriz era 2D, que se mantenga 2D y no pase a ser 1D.

Veamos estos puntos con un ejemplo:

In [8]:
A = np.array([[1, 2], [3, 4]])
print("Dimensiones de A: ", A.shape, "\n")
print('axis=1 y keepdims=True')
print(np.sum(A, axis=1, keepdims=True))
print(np.sum(A, axis=1, keepdims=True).shape, "\n")
print('axis=1 y keepdims=False')
print(np.sum(A, axis=1, keepdims=False))
print(np.sum(A, axis=1, keepdims=False).shape, "\n")
print('axis=0 y keepdims=True')
print(np.sum(A, axis=0, keepdims=True))
print(np.sum(A, axis=0, keepdims=True).shape, "\n")
print('axis=0 y keepdims=False')
print(np.sum(A, axis=0, keepdims=False))
print(np.sum(A, axis=0, keepdims=False).shape)

Dimensiones de A:  (2, 2) 

axis=1 y keepdims=True
[[3]
 [7]]
(2, 1) 

axis=1 y keepdims=False
[3 7]
(2,) 

axis=0 y keepdims=True
[[4 6]]
(1, 2) 

axis=0 y keepdims=False
[4 6]
(2,)


<a name='3-4-1'></a>
#### 3.4.1 - Derivadas de las funciones de activación

Las derivadas de las funciones sigmoide y ReLU ya se han vista en la teoría. Si $g^{[l]}()$ es la función de activación, `sigmoide_backward()` y `relu_backward()` están calculando:

$$dZ^{[l]} = dA^{[l]} * g^{[l]}{'}(Z^{[l]}) \tag{5}$$  

In [9]:
def relu_backward(dA, cache):
    """
    Implementa la propagación hacia atrás para una unica unidad ReLU.

    Argumentos:
    dA -- gradiente post-activación, de cualquier dimension.
    cache -- Es la variable 'Z' que fue almacenada en 'cache' para hacer un calculo de la retropropagacion eficiente.

    Devuelve:
    dZ -- Gradiente del coste con respecto a Z.
    """
    
    Z = cache
    # Hacemos una copia profunda de dA con 'copy=True', argumento que crea una copia completa de un objeto y todos sus elementos.
    # En el caso de una matriz, esto significa que se crea una nueva matriz con los mismos valores que la matriz original, 
    # pero en una ubicación de memoria diferente. Esto significa que si cambias un valor en la matriz original, 
    # no afectará a la copia profunda.
    dZ = np.array(dA, copy=True)
    
    # Cuando Z = 0, podemos decidir si queremos que equivalga a 0 o a 1. En este caso, a 0.
    dZ[Z <= 0] = 0
    
    # Clave asegurar que las dimensiones son correctas
    assert (dZ.shape == Z.shape)
    
    return dZ

In [10]:
def sigmoide_backward(dA, cache):
    """
    Implementa la propagación hacia atrás para una unica unidad ReLU.

    Argumentos:
    dA -- gradiente post-activación, de cualquier dimension.
    cache -- Es la variable 'Z' que fue almacenada en 'cache' para hacer un calculo de la retropropagacion eficiente.

    Devuelve:
    dZ -- Gradiente del coste con respecto a Z.
    """
    
    Z = cache
    # Sigmoide va a ser la ultima capa. dZ es la derivada del coste con respecto a Z de la ultima capa, es decir, dJ/dZ y, 
    # segun la regla de la cadena, es equivalente a:
    #       dZ = dJ/dZ = dJ/dA * dA/dZ, donde dJ/dA = dA, y donde dA/dZ = derivada de la funcion 'sigmoide(Z)', 
    # y la derivada del sigmoide es: s'(z) = s(z)*(1-s(z))
    # Funcion sigmoide
    s = 1 / (1 + np.exp(-Z))
    # dJ/dZ
    dZ = dA * s * (1 - s)
    
    # Clave asegurar que las dimensiones son correctas
    assert (dZ.shape == Z.shape)
    
    return dZ

<a name='3-4-2'></a>
#### 3.4.2 - Cálculo de las derivadas de la función lineal

Para la capa $l$, la parte lineal es la ecuación (1) seguida de una activación. Supongamos que ya hemos calculado la derivada $dZ^{[l]} = \frac{\partial \mathcal{J} }{\partial Z^{[l]}}$. A continuación tenemos que hallar $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$. Las tres derivadas parciales se calculan utilizando $dZ^{[l]}$ con las siguientes fórmulas:

$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1]^T} \tag{6}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{7}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{J} }{\partial A^{[l-1]}} = W^{[l]^T} dZ^{[l]} \tag{8}$$


$A^{[l-1]^T}$ es la traspuesta de $A^{[l-1]}$. Implementamos estas tres ecuaciones para obtener la función `backward_lineal()`.

In [11]:
def backward_lineal(dZ, cache):
    """
    Implementa la parte lineal de la retropropagación para una sola capa (capa l)

    Argumentos:
    dZ -- Gradiente del coste con respecto a la salida lineal (de la capa l actual).
    cache -- tupla de valores (A_prev, W, b) procedentes de forward_lineal() en la capa actual

    Devuelve:
    dA_prev -- Gradiente del coste con respecto a la activación (de la capa anterior l-1); mismas dimensiones que A_prev
    dW -- Gradiente del coste con respecto a W (capa actual l), mismas dimensiones que W
    db -- Gradiente del coste con respecto a b (capa actual l), mismas dimensiones que b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = np.dot(dZ, A_prev.T)/m
    db = np.sum(dZ, axis=1, keepdims=True)/m
    dA_prev = np.dot(W.T, dZ)    
    
    # Aseguramos que las dimensiones sean correctas
    assert(dA_prev.shape == A_prev.shape)
    assert(dW.shape == W.shape)
    assert(db.shape == b.shape)    
    
    return dA_prev, dW, db

<a name='3-4-3'></a>
#### 3.4.3 - Cálculo de las derivadas de las funciones de activación

Por último, se implementa la retropropagación para la capa $l$, el paso *LINEAL->ACTIVACION*. Agrupamos el cálculo lineal con las derivadas de las funciones de activación en una única función. Para usarlas, realizamos las siguientes llamadas:

```python
dZ = sigmoide_backward(dA, activacion_cache)

dZ = relu_backward(dA, activacion_cache)
```

In [12]:
def backward_activacion_lineal(dA, cache, activacion):
    """
    Implementar la retropropagacion para la capa de la forma LINEAL->ACTIVACION
    
    Argumentos:
    dA -- gradiente post-activación para la capa actual l 
    cache -- tupla de valores (cache_lineal, activacion_cache) almacenados para calcular la propagación hacia atrás eficientemente
    activation -- la funcion de activacion a utilizar en esta capa, almacenada como cadena de texto: "sigmoide" o "relu".
    
    Devuelve
    dA_prev -- Gradiente del coste con respecto a la activación (de la capa anterior l-1); mismas dimensiones que A_prev
    dW -- Gradiente del coste con respecto a W (capa actual l), mismas dimensiones que W
    db -- Gradiente del coste con respecto a b (capa actual l), mismas dimensiones que b
    """
    cache_lineal, activacion_cache = cache
    
    if activacion == "relu":
        dZ = relu_backward(dA, activacion_cache)
        dA_prev, dW, db = backward_lineal(dZ, cache_lineal)
    elif activacion == "sigmoide":
        dZ = sigmoide_backward(dA, activacion_cache)
        dA_prev, dW, db = backward_lineal(dZ, cache_lineal)
    else:
        print("¡Error! Solo se admiten relu o sigmoide como parametros en \"activacion\"")
    
    return dA_prev, dW, db

<a name='3-5'></a>
### 3.5 - Actualización de los parámetros

Actualizamos los parámetros del modelo aplicando el algoritmo del descenso del gradiente:

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{9}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{10}$$

donde $\alpha$ es la tasa de aprendizaje. 

Tras calcularlos, se guardarán en un diccionario, como se hizo en el código del archivo de una red neuronal de una capa. Aunque este es el apartado de la red de dos capas, para no crear dos funciones distintas se crea diréctamente la función general de actualización para una red de L capas, es decir, se actualizarán los parámetros $W^{[l]}$ y $b^{[l]}$ para $l = 1, 2, ..., L$.

In [13]:
def actualizar_parametros(parametros_entrada, gradientes, tasa_aprendizaje):
    """
    Actualiza los parámetros utilizando la regla de actualización por descenso de gradiente.
    
    Argumentos:
    parametros_entrada -- diccionario python que contiene los parametros 
    gradientes -- diccionario python que contiene los gradientes
    tasa_aprendizaje -- hiperparametro que representa la tasa de aprendizaje utilizada en la regla de actualizacion
    
    Devuelve:
    parametros -- diccionario python que contiene los parametros actualizados 
                  parametros["W" + str(l)] = ... 
                  parametros["b" + str(l)] = ...
    """
    # Recupera una copia de cada parámetro del diccionario "parametros_entrada".
    parametros = parametros_entrada.copy()
    # numero de capas de la red neuronal (para cada l-esima capa hay un parametro W y otro b)
    L = len(parametros) // 2 

    # Aplicar regla de actualizacion a cada parametro
    for l in range(L):
        parametros["W" + str(l + 1)] = parametros["W" + str(l + 1)] - tasa_aprendizaje * gradientes["dW" + str(l + 1)]
        parametros["b" + str(l + 1)] = parametros["b" + str(l + 1)] - tasa_aprendizaje * gradientes["db" + str(l + 1)]        

    return parametros

<a name='4'></a>
## 4 - Modelo de red neuronal con L capas (L-1 capas ocultas)

<a name='4-1'></a>
### 4.1 - Inicialización de los parámetros

La inicialización de una red neuronal de L capas es más complicada que lo visto hasta ahora, debido a que hay más matrices de pesos y vectores de sesgos. Al implementar la función `inicializacion_profunda()`, es importante asegurarse que las dimensiones sean coherentes entre capas.

> Nota: Recuérdese que $n^{[l]}_h$ es el número de unidades (ocultas) en la capa $l$. Por ejemplo, si las dimensiones de la matriz de entrenamiento, $X$, son $(12288, 209)$ (siendo $m=209$ el numero de casos/ejemplos de entrenamiento) entonces:

<table style="width:100%">
    <tr>
        <td>  </td> 
        <td> <b>Dimensiones de W</b> </td> 
        <td> <b>Dimensiones de b</b>  </td> 
        <td> <b>Activacion lineal</b> </td>
        <td> <b>Dimensiones de la activación lineal</b> </td> 
    <tr>
    <tr>
        <td> <b>Capa 1</b> </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> <b>Capa 2</b> </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>  
   <tr>
       <td> <b>Capa L-1</b> </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
   <tr>
   <tr>
       <td> <b>Capa L</b> </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>
</table>

**Importante**: Se menciona durante todo el trabajo que, cuando se calcula $W X + b$ en Python, se ejecuta el **_broadcasting_** para vectores y matrices. Por ejemplo, si: 

$$ W = \begin{bmatrix}
    w_{00}  & w_{01} & w_{02} \\
    w_{10}  & w_{11} & w_{12} \\
    w_{20}  & w_{21} & w_{22} 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    x_{00}  & x_{01} & x_{02} \\
    x_{10}  & x_{11} & x_{12} \\
    x_{20}  & x_{21} & x_{22} 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    b_0  \\
    b_1  \\
    b_2
\end{bmatrix}$$

Entonces $WX + b$ sería:

$$ WX + b = \begin{bmatrix}
    (w_{00}x_{00} + w_{01}x_{10} + w_{02}x_{20}) + b_0 & (w_{00}x_{01} + w_{01}x_{11} + w_{02}x_{21}) + b_0 & \cdots \\
    (w_{10}x_{00} + w_{11}x_{10} + w_{12}x_{20}) + b_1 & (w_{10}x_{01} + w_{11}x_{11} + w_{12}x_{21}) + b_1 & \cdots \\
    (w_{20}x_{00} + w_{21}x_{10} + w_{22}x_{20}) + b_2 &  (w_{20}x_{01} + w_{21}x_{11} + w_{22}x_{21}) + b_2 & \cdots
\end{bmatrix} $$


Consideraciones:

- La estructura del modelo es *[LINEAL -> RELU] $ \times$ (L-1) -> LINEAL -> SIGMOIDE*. Es decir, tiene $L-1$ capas utilizando la función de activación ReLU seguidas de una capa de salida con una función de activación sigmoide.
- Ya no se utilizará una inicialización aleatoria de la matriz de pesos, para una red de L-capas es insuficiente y además con activaciones ReLU no da buenos resultados. Se utilizará la **inicialización Xavier**, que usa un factor de escala `sqrt(1./layers_dims[l-1])` para los pesos $W^{[l]}$.
- Se utilizará una inicialización con ceros para los sesgos: `np.zeros()`. La documentación para [np.zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html).
- Se almacenará $n^{[l]}_h$, el número de unidades en las diferentes capas, en una varible `dims_capas`. 
    - Por ejemplo, esta variable en el archivo anterior donde se clasificaban los datos de un plano sería [2,4,1]: había dos características de entrada, una capa oculta con cuatro unidades ocultas (en la ejecución inicial), y una capa de salida con una unidad. Los parámetros tenían, por tanto, las siguientes dimensiones: `W1` -> (4,2), `b1` -> (4,1), `W2` -> (1,4) y `b2` -> (1,1). Esto es lo que se va a generalizar a continuación.

In [14]:
def inicializacion_profunda(dims_capas):
    """
    Argumentos:
    dims_capas -- array python (lista) que contiene las dimensiones de cada capa de nuestra red.
    
    Devuelve:
    parametros -- diccionario python que contiene los parámetros "W1", "b1", ..., "WL", "bL":
                    Wl -- matriz de pesos de dimensiones (dims_capas[l], dims_capas[l-1])
                    bl -- vector de sesgo de dimensiones (dims_capas[l], 1)
    """
    
    np.random.seed(1)
    parametros = {}
    # Numero de capas en la red neuronal
    L = len(dims_capas)

    for l in range(1, L):
        # Multiplicar por 0.01 ya no es suficiente con un modelo mas grande. Utilizamos la inicializacion Xavier
        parametros['W' + str(l)] = np.random.randn(dims_capas[l], dims_capas[l - 1])  / np.sqrt(dims_capas[l-1]) #*0.01
        parametros['b' + str(l)] = np.zeros((dims_capas[l], 1))        
        
        # Asegurar que las dimensiones son correctas
        assert(parametros['W' + str(l)].shape == (dims_capas[l], dims_capas[l - 1]))
        assert(parametros['b' + str(l)].shape == (dims_capas[l], 1))
        
    return parametros

<a name='4-2'></a>
### 4.2 - Propagación hacia delante (_forward propagation_)

Combinamos las dos funciones que se utilizaron en la red de dos capas en una tercera (tercer paso) con esta estructura:

- [LINEAL -> RELU] $\times$ (L-1) -> LINEAL -> SIGMOIDE

Esta función con validez general llama a la función `forward_activacion_lineal(activacion="relu")` $L-1$ veces y por último llama a la misma función pero con activación sigmoide: `forward_activacion_lineal(activacion="sigmoide")`.

<img src="imagenes/propagacion_hacia_delante.png" style="width:600px;height:300px;">
<caption><center> <b>Figura 3</b> : Modelo [LINEAL -> RELU] $\times$ (L-1) -> LINEAL -> SIGMOIDE</center></caption><br>

> Nota: En el codigo de la función, la variable `AL` particulariza la ecuación (2) para la activación sigmoide en la capa L, la última, e indica $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$. Y, como se ha señalado constantemente en el trabajo, la salida de la activación de la útlima capa equivale a $\hat{Y}$, es decir, la predicción final.

In [15]:
def propagacion_L_capas(X, parametros):
    """
    Implementa la propagación hacia delante para el cálculo [LINEAL->RELU]*(L-1)->LINEAL->SIGMOIDE
    
    Argumentos:
    X -- datos, array numpy de dimensiones (caracteristicas por ejemplo, número de ejemplos)
    parametros -- salida de inicializacion_profunda()
    
    Devuelve:
    AL -- valor de activación de la capa de salida
    caches -- lista de caches que contienen cada cache de forward_activacion_lineal() (hay L, indexadas de 0 a L-1)
    """

    caches = []
    # La activacion inicial, A[0], es la capa de entrada
    A = X
    # numero de capas de la red neuronal (para cada l-esima capa hay un parametro W y otro b)
    L = len(parametros) // 2
    
    # Implementar [LINEAL -> RELU]*(L-1). Añadimos "cache" a la lista de "caches".
    # El bucle comienza en 1 porque la capa 0 es la de entrada
    for l in range(1, L):
        A_prev = A 
        A, cache = forward_activacion_lineal(A_prev, 
                                             parametros['W' + str(l)], 
                                             parametros['b' + str(l)], 
                                             activacion='relu')
        caches.append(cache)        
    
    # Implementar LINEAL -> SIGMOIDE. Añadimos "cache" a la lista de "caches".
    AL, cache = forward_activacion_lineal(A, 
                                          parametros['W' + str(L)], 
                                          parametros['b' + str(L)], 
                                          activacion='sigmoide')
    caches.append(cache)
    
    # Nos aseguramos que las dimensiones sean correctas (n_y, m)
    assert(AL.shape == (1,X.shape[1]))
    
    return AL, caches

Con esto se habría implementado un proceso de propagación hacia delante completo que toma la matriz de entrada, $X$, y devuelve un vector $A^{[L]}$ con las predicciones. También lleva un registro de todos los valores intermedios calculados en "caches".

<a name='4-3'></a>
### 4.3 - Cálculo del coste global

Igual que en el apartado [3.3 - Cálculo del coste global](#3-3), utilizando $A^{[L]}$ se puede calcular el coste de las predicciones de la red neuronal.

<a name='4-4'></a>
### 4.4 - Retropropagación (_backward propagation_)

A la hora de implementar la retropropagación, utilizaremos la misma lógica que para el modelo de dos capas, donde se aprovecha la "cache" almacenada. En cada iteración de la función `propagacion_L_capas()` se guarda un cache que contiene la tupla de valores (Z, A, W, b). En la retropropagación, se utilizan esas variables para calcular los gradientes. Por lo tanto, en la función que se crea a continuación, `retropropagacion_L_capas()`, se itera sobre toda las capas ocultas, empezando por la capa final, $L$, hasta la capa $1$. En cada paso, se utiliza el "cache" correspondiente a la capa $l$ para realizar la retropropagación (las derivadas) sobre la capa $l$. Este procedimiento se esquematiza en la figura 4.

<img src="imagenes/retropropagacion.png" style="width:450px;height:300px;">
<caption><center><b>Figura 4</b>: Retropropagación completa en un modelo de L capas.</center></caption>

Para **inicializar la retropropagación** empezando por la última capa, sabemos que la activación es una función sigmoide, con salida $A^{[L]} = \sigma(Z^{[L]})$. Por lo tanto el primer paso es calcular `dAL` $= \frac{\partial \mathcal{J}}{\partial A^{[L]}}$. Si la salida es sigmoide el valor de esta derivada, vectorizado a todos los ejemplos de la red, es:

$$\frac{\partial \mathcal{J}}{\partial A^{[L]}} = -\left(\frac{Y}{A^{[L]}} - \frac{1 - Y}{1- A^{[L]}}\right)\tag{11} $$

En python se puede aplicar de la siguiente manera:
```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivada del coste con respecto a AL
```

A continuación se utiliza este gradiente de post-activación `dAL` para continuar hacar atrás en las capas. Como se ve en la figura 3, lo siguiente sería pasar del sigmoide a la derivada de la función lineal y, para ello, nos valdremos de la función `sigmoide_backward()`. El "cache" que utilizará esta función será el correspondiente, almacenado durante la ejecución de `propagacion_L_capas()`. Con esto se termina el bloque la combinación [LINEAL -> SIGMOIDE_backward]

Tras esto, se utilizará un bucle `for` para iterar la retropropagación en las otras capas utilizando la combinación la combinación [LINEAL -> RELU_backward]. Los valores para cada capa de `dA`, `dW`, y `db` se almacenarán en un diccionario utilizando la siguiente fórmula:

$$gradientes["dW" + str(l)] = dW^{[l]}\tag{12} $$

Por ejemplo, para $l=4$ se guardaría $dW^{[4]}$ en `gradientes["dW4"]`.

En definitiva, se van a combinar las dos funciones que se utilizaron en la red de dos capas en una tercera (tercer paso) con esta estructura:

LINEAL -> SIGMOIDE backward -> [LINEAL -> RELU backward] $\times$ (L-1)

In [16]:
def retropropagacion_L_capas(AL, Y, caches):
    """
    Implementa la retropropagacion para el grupo [LINEAL->RELU] * (L-1) -> LINEAL -> SIGMOIDE
    
    Argumentos:
    AL -- vector de predicciones, salida de la propagación hacia delante (propagacion_L_capas())
    Y -- Vector de etiquetas verdaderas de dimensiones (1, número de casos de entrenamiento).
    caches -- lista del cache de cada capa. Contiene:
                cada cache linear_activation_forward("relu") (es caches[l], para l en range(L-1), es decir, l = 0...L-2)
                el cache de linear_activation_forward("sigmoide") (es caches[L-1])
    
    Devuelve:
    gradientes -- Un diccionario con los gradientes
                  gradientes["dA" + str(l)] 
                  gradientes["dW" + str(l)]
                  gradientes["db" + str(l)]
    """
    gradientes = {}
    # Numero de capas
    L = len(caches)
    m = AL.shape[1]
    # Tras esta linea, Y tiene las mismas dimensiones que AL.
    Y = Y.reshape(AL.shape)
    
    # Inicializar la retropropagacion
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Capa L: gradientes de (SIGMOID -> LINEAL). Entrada: "dAL, cache_actual". 
    #                                            Salidas: "gradientes["dAL-1"], gradientes["dWL"], gradientes["dbL"]
    # la capa L es el ultimo valor de la lista "caches", que contiene L elementos, y al empezar el indice por 0, el ultimo es L-1
    cache_actual = caches[L-1] # tambien cache_actual = caches[-1] nos lleva al ultimo elemento
    dA_prev_temp, dW_temp, db_temp = backward_activacion_lineal(dAL, cache_actual, activacion = "sigmoide")
    gradientes["dA" + str(L-1)] = dA_prev_temp
    gradientes["dW" + str(L)] = dW_temp
    gradientes["db" + str(L)] = db_temp
    
    # Bucle de l=L-2 a l=0 para implementar los gradientes de (RELU -> LINEAL). range(L-1) va de 0 a L-2
    for l in reversed(range(L-1)):
        # Entradas: "gradientes["dA" + str(l + 1)], cache_actual". 
        # Salidas: "gradientes["dA" + str(l)] , gradientes["dW" + str(l + 1)] , gradientes["db" + str(l + 1)]
        # La suma (l + 1) es por el hecho de que nuestro indice va de (0, L-1) y no de (1, L), por lo que la equivalencia  
        # entre el bucle y las capas se logra sumando 1.
        cache_actual = caches[l]
        dA_prev_temp, dW_temp, db_temp = backward_activacion_lineal(gradientes["dA" + str(l + 1)], 
                                                                    cache_actual, 
                                                                    activacion = "relu")
        # Otra forma equivalente, que es lo que tenemos encapsulado en 2 lineas en la funcion backward_activacion_lineal():
        # dA_prev_temp, dW_temp, db_temp = backward_lineal(relu_backward(grads["dA" + str(l + 1)], 
        #                                                  current_cache[1]), current_cache[0])
        gradientes["dA" + str(l)] = dA_prev_temp
        gradientes["dW" + str(l + 1)] = dW_temp
        gradientes["db" + str(l + 1)] = db_temp        

    return gradientes

<a name='4-5'></a>
### 4.5 - Actualización de los parámetros

Igual que en el apartado [3.5 - Actualización de los parámetros](#3-5).